In [2]:
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.read_csv('./dataset/tmdb_5000_movies.csv', header=0) 
print( df1.shape )
print( df1['title'][1] )
print( df1.head(2) )

(4803, 20)
Pirates of the Caribbean: At World's End
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   

                                       homepage     id  \
0                   http://www.avatarmovie.com/  19995   
1  http://disney.go.com/disneypictures/pirates/    285   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   

 

In [3]:
df2 = df1[ ['id', 'genres', 'vote_average', 'vote_count', 'popularity', 'title', 'keywords', 'overview'] ]
print( df2.shape )
print( df2[['id', 'vote_average', 'vote_count']][:3]  )

(4803, 8)
       id  vote_average  vote_count
0   19995           7.2       11800
1     285           6.9        4500
2  206647           6.3        4466


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

print( df2[['genres']].head(3) )
count_vector = CountVectorizer( ngram_range=(1,3) )
c_vector_genres = count_vector.fit_transform( df2['genres'] )
print( c_vector_genres.shape )
print( c_vector_genres[0] )
print( "add1: ")
print( c_vector_genres[0, 0:40] )
print( "add2: ")
print( c_vector_genres[0, 0], c_vector_genres[0, 1],  c_vector_genres[0, 33], c_vector_genres[0, 255])
print( c_vector_genres[1, 0], c_vector_genres[1, 1],  c_vector_genres[1, 2])

                                              genres
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
(4803, 440)
  (0, 255)	4
  (0, 33)	1
  (0, 331)	4
  (0, 60)	1
  (0, 18)	1
  (0, 77)	1
  (0, 21)	1
  (0, 192)	1
  (0, 51)	1
  (0, 391)	1
  (0, 207)	1
  (0, 278)	1
  (0, 34)	1
  (0, 332)	1
  (0, 61)	1
  (0, 268)	1
  (0, 19)	1
  (0, 334)	1
  (0, 78)	1
  (0, 270)	1
  (0, 22)	1
  (0, 348)	1
  (0, 193)	1
  (0, 290)	1
  (0, 52)	1
  (0, 362)	1
  (0, 392)	1
  (0, 279)	1
  (0, 35)	1
  (0, 333)	1
  (0, 65)	1
  (0, 269)	1
  (0, 20)	1
  (0, 335)	1
  (0, 82)	1
  (0, 271)	1
  (0, 23)	1
  (0, 349)	1
  (0, 205)	1
  (0, 291)	1
  (0, 53)	1
  (0, 363)	1
add1: 
  (0, 33)	1
  (0, 18)	1
  (0, 21)	1
  (0, 34)	1
  (0, 19)	1
  (0, 22)	1
  (0, 35)	1
  (0, 20)	1
  (0, 23)	1
add2: 
0 0 1 4
0 0 0


In [63]:
from sklearn.metrics.pairwise import cosine_similarity

genres_c_sim = cosine_similarity( c_vector_genres )
print( genres_c_sim )

genres_c_sim = genres_c_sim.argsort()[:, ::-1]
print( genres_c_sim )

[[1.         0.87841046 0.755433   ... 0.44444444 0.         0.31426968]
 [0.87841046 1.         0.75555556 ... 0.42163702 0.         0.2981424 ]
 [0.755433   0.75555556 1.         ... 0.42163702 0.         0.2981424 ]
 ...
 [0.44444444 0.42163702 0.42163702 ... 1.         0.         0.31426968]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.31426968 0.2981424  0.2981424  ... 0.31426968 0.         1.        ]]
[[   0  870   46 ... 4400 4674 4571]
 [ 330  129 2390 ... 4681 4293 4633]
 [1740 1542    2 ... 4657 4674 4504]
 ...
 [4800 3809 3805 ... 4657 4622 4105]
 [4802 1594 1596 ... 3204 3205    0]
 [4802 4002 4616 ... 4633 4550 4714]]


In [61]:
def get_recommend_movie_list(df, input_title, top=5):
    # 입력 제목 영화의 인텍스 값 추출
    target_index = df[ df['title'] == input_title].index.values
    print( target_index )
    # 코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 추출    
    sim_index1 = genres_c_sim[target_index, :top]
    print( sim_index1 )
    sim_index2 = sim_index1.reshape(-1)
    print( sim_index2 )
    # 본인 제외
    sim_index3 = sim_index2[ sim_index2 != target_index ]
    print( sim_index3 )
    # 최종 선택   
    return  df.iloc[ sim_index3 ]

get_recommend_movie_list( df2, 'Pirates of the Caribbean: At World's End' )

[0]
[[   0  870   46 3494   14]]
[   0  870   46 3494   14]
[ 870   46 3494   14]


,id,genres,vote_average,vote_count,popularity,title,keywords,overview
870,8536,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.5,629,30.515175,Superman II,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",Three escaped criminals from the planet Krypto...
46,127585,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.5,6032,118.078691,X-Men: Days of Future Past,"[{""id"": 1228, ""name"": ""1970s""}, {""id"": 1852, ""...",The ultimate X-Men ensemble fights a war for t...
3494,27549,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4.6,17,1.478505,Beastmaster 2: Through the Portal of Time,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","Mark Singer returns as Dar, the warrior who ca..."
14,49521,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.5,6359,99.398009,Man of Steel,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",A young boy learns that he has extraordinary p...


In [ ]:
count_vector = CountVectorizer( ngram_range=(1,3) )
c_vector_genres = count_vector.fit_transform( df2['genres'] )

genres_c_sim = cosine_similarity( c_vector_genres )
genres_c_sim = genres_c_sim.argsort()[:, ::-1]